In [182]:
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score

In [168]:
df = pd.read_csv("dataproject2025.csv")

In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086683 entries, 0 to 1086682
Data columns (total 39 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1086683 non-null  int64  
 1   issue_d                  1086236 non-null  float64
 2   loan duration            1086236 non-null  float64
 3   annual_inc               1086236 non-null  float64
 4   avg_cur_bal              1086236 non-null  float64
 5   bc_open_to_buy           1086236 non-null  float64
 6   bc_util                  1086236 non-null  float64
 7   delinq_2yrs              1086236 non-null  float64
 8   dti                      1086236 non-null  float64
 9   emp_length               1086236 non-null  object 
 10  emp_title                1086236 non-null  object 
 11  fico_range_high          1086236 non-null  float64
 12  funded_amnt              1086236 non-null  float64
 13  grade                    1086236 non-null 

In [169]:
df.head()

,Unnamed: 0,issue_d,loan duration,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,dti,emp_length,...,purpose,revol_bal,revol_util,sub_grade,target,tax_liens,zip_code,Pct_afro_american,Predictions,Predicted probabilities
0,0,2013.0,0.0,39600.0,1379.0,21564.0,16.1,0.0,2.49,2 years,...,home_improvement,4136.0,16.1,B2,0.0,0.0,782.0,7.388592,0.0,0.053051
1,1,2013.0,0.0,55000.0,9570.0,16473.0,53.9,0.0,22.87,10+ years,...,debt_consolidation,36638.0,61.2,B2,0.0,0.0,481.0,9.745456,0.0,0.084507
2,2,2013.0,0.0,325000.0,53306.0,13901.0,67.1,0.0,18.55,5 years,...,debt_consolidation,29581.0,54.6,A3,0.0,0.0,945.0,7.542862,0.0,0.037206
3,3,2013.0,0.0,130000.0,36362.0,3567.0,93.0,0.0,13.03,10+ years,...,debt_consolidation,10805.0,67.0,B3,0.0,0.0,809.0,6.598132,0.0,0.061371
4,4,2013.0,1.0,73000.0,24161.0,4853.0,74.7,1.0,23.13,6 years,...,debt_consolidation,27003.0,82.8,D5,1.0,0.0,802.0,7.058900,1.0,0.345896


In [170]:
df.isna().sum()

Unnamed: 0                   0
issue_d                    447
loan duration              447
annual_inc                 447
avg_cur_bal                447
bc_open_to_buy             447
bc_util                    447
delinq_2yrs                447
dti                        447
emp_length                 447
emp_title                  447
fico_range_high            447
funded_amnt                447
grade                      447
home_ownership             447
inq_last_6mths             447
int_rate                   447
mo_sin_old_rev_tl_op       447
mo_sin_rcnt_rev_tl_op      447
mo_sin_rcnt_tl             447
mort_acc                   447
mths_since_recent_bc       447
num_actv_bc_tl             447
num_bc_tl                  447
num_il_tl                  447
num_rev_accts              447
open_acc                   447
pub_rec                    447
pub_rec_bankruptcies       447
purpose                    447
revol_bal                  447
revol_util                 447
sub_grad

In [171]:
feature_name_map = {
    "Unnamed: 0": "id",
    # "annual_inc": "annual_income",
    # "avg_cur_bal": "avg_current_balance",
    # "bc_open_to_buy": "bankcard_open_to_buy",
    # "bc_util": "bankcard_utilization",
    # "delinq_2yrs": "delinquencies_2yrs",
    # "dti": "debt_to_income_ratio",
    # "emp_length": "employment_length_years",
    # "emp_title": "employment_title",
    # "fico_range_high": "fico_score_high",
    # "funded_amnt": "funded_amount",
    # "grade": "loan_grade",
    # "home_ownership": "home_ownership_status",
    # "inq_last_6mths": "inquiries_last_6mths",
    # "int_rate": "interest_rate",
    # "issue_d": "issue_year",
    # "loan duration": "loan_duration_60m_flag",
    # "mo_sin_old_rev_tl_op": "months_since_oldest_revolving",
    # "mo_sin_rcnt_rev_tl_op": "months_since_recent_revolving",
    # "mo_sin_rcnt_tl": "months_since_recent_account",
    # "mort_acc": "num_mortgage_accounts",
    # "mths_since_recent_bc": "months_since_recent_bankcard",
    # "num_actv_bc_tl": "num_active_bankcards",
    # "num_bc_tl": "num_bankcard_accounts",
    # "num_il_tl": "num_installment_accounts",
    # "num_rev_accts": "num_revolving_accounts",
    # "open_acc": "num_open_accounts",
    # "pub_rec": "num_public_records",
    # "pub_rec_bankruptcies": "num_bankruptcies",
    # "purpose": "loan_purpose",
    # "revol_bal": "revolving_balance",
    # "revol_util": "revolving_utilization",
    # "sub_grade": "loan_subgrade",
    # "target": "default_flag",
    # "tax_liens": "num_tax_liens",
    # "zip_code": "zipcode_prefix",
    # "Pct_afro_american": "pct_african_american_zip",
    # "Predictions": "predicted_default_flag",
    # "Predicted probabilities": "predicted_default_probability"
}

df.rename(columns=col_map, inplace=True)

In [172]:
na_rows = df.isna().any(axis=1)
print(f"Number of rows with NA: {na_rows.sum()}")
display(df[na_rows])

print(df.isna().sum(axis=1).value_counts())

Number of rows with NA: 894


,id,issue_d,loan duration,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,dti,emp_length,...,purpose,revol_bal,revol_util,sub_grade,target,tax_liens,zip_code,Pct_afro_american,Predictions,Predicted probabilities
1085789,1086236,2017.0,0.0,105000.00,31252.0,5852.0,82.3,0.0,17.47,10+ years,...,debt_consolidation,27872.0,75.3,B4,0.0,0.0,891.0,11.886387,NaN,NaN
1085790,1086237,2017.0,0.0,60000.00,878.0,12391.0,6.1,0.0,6.06,5 years,...,other,809.0,5.3,B1,0.0,0.0,400.0,4.462141,NaN,NaN
1085791,1086238,2017.0,1.0,110000.00,17106.0,1440.0,94.5,1.0,22.28,10+ years,...,credit_card,25054.0,89.8,C1,0.0,0.0,802.0,7.058900,NaN,NaN
1085792,1086239,2017.0,0.0,181115.38,65412.0,53449.0,22.0,0.0,3.42,10+ years,...,home_improvement,65194.0,20.1,A1,0.0,0.0,957.0,5.658004,NaN,NaN
1085793,1086240,2017.0,1.0,85000.00,5317.0,3083.0,89.0,4.0,16.14,2 years,...,debt_consolidation,9674.0,86.0,C1,1.0,0.0,334.0,18.539195,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086678,1068020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.179364
1086679,1068750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.078607
1086680,1069795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.070936
1086681,1070263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.326686


0     1085789
2         447
36        447
Name: count, dtype: int64


In [173]:
# these complement

na2 = df[df.isna().sum(axis=1) == 2]
display(na2)
na36 = df[df.isna().sum(axis=1) == 36]
display(na36)

print(f'Union: {len(set(na2["id"]).union(set(na36["id"])))}')
print(f'Intersection: {len(set(na2["id"]).intersection(set(na36["id"])))}')

,id,issue_d,loan duration,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,dti,emp_length,...,purpose,revol_bal,revol_util,sub_grade,target,tax_liens,zip_code,Pct_afro_american,Predictions,Predicted probabilities
1085789,1086236,2017.0,0.0,105000.00,31252.0,5852.0,82.3,0.0,17.47,10+ years,...,debt_consolidation,27872.0,75.3,B4,0.0,0.0,891.0,11.886387,NaN,NaN
1085790,1086237,2017.0,0.0,60000.00,878.0,12391.0,6.1,0.0,6.06,5 years,...,other,809.0,5.3,B1,0.0,0.0,400.0,4.462141,NaN,NaN
1085791,1086238,2017.0,1.0,110000.00,17106.0,1440.0,94.5,1.0,22.28,10+ years,...,credit_card,25054.0,89.8,C1,0.0,0.0,802.0,7.058900,NaN,NaN
1085792,1086239,2017.0,0.0,181115.38,65412.0,53449.0,22.0,0.0,3.42,10+ years,...,home_improvement,65194.0,20.1,A1,0.0,0.0,957.0,5.658004,NaN,NaN
1085793,1086240,2017.0,1.0,85000.00,5317.0,3083.0,89.0,4.0,16.14,2 years,...,debt_consolidation,9674.0,86.0,C1,1.0,0.0,334.0,18.539195,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086231,1086678,2017.0,1.0,100000.00,28408.0,19038.0,14.2,0.0,8.32,< 1 year,...,debt_consolidation,3162.0,14.2,C3,0.0,0.0,612.0,7.501640,NaN,NaN
1086232,1086679,2017.0,1.0,107000.00,5528.0,20859.0,26.0,3.0,11.65,< 1 year,...,other,9688.0,24.9,E2,1.0,1.0,801.0,1.493616,NaN,NaN
1086233,1086680,2017.0,0.0,65000.00,3724.0,26229.0,24.6,1.0,19.55,10+ years,...,debt_consolidation,9751.0,15.7,A5,0.0,0.0,187.0,6.371972,NaN,NaN
1086234,1086681,2017.0,0.0,41000.00,3275.0,7651.0,5.5,1.0,19.99,5 years,...,credit_card,1356.0,10.1,B4,0.0,0.0,105.0,11.586467,NaN,NaN


,id,issue_d,loan duration,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,dti,emp_length,...,purpose,revol_bal,revol_util,sub_grade,target,tax_liens,zip_code,Pct_afro_american,Predictions,Predicted probabilities
1086236,2276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.276209
1086237,5641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.196808
1086238,6398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.126141
1086239,7060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.143869
1086240,7100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.030056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086678,1068020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.179364
1086679,1068750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.078607
1086680,1069795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.070936
1086681,1070263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.326686


Union: 894
Intersection: 0


In [174]:
#ProfileReport(df)

In [175]:
df_dropped = df.dropna(axis=0) # either y misses or all X miss


In [176]:
def get_features(df: pd.DataFrame()) -> pd.DataFrame():
    """
    Get additional features.
    """        

    df_with_features = (
        df
        .assign(

            # logs --> to money, not months or so
            annual_inc_log = np.log1p(df["annual_inc"]),
            avg_cur_bal_log = np.log1p(df["avg_cur_bal"]),
            fico_range_high_log = np.log1p(df["fico_range_high"]),
            revol_bal_log = np.log1p(df["revol_bal"]),

            # broader zip code area
            zip_code2 = np.round(df["zip_code"] / 10, 0),

            # total balance?
            cur_balance = df["avg_cur_bal"] * df["open_acc"],

            # flags
            delinq_2yrs_flag = df["delinq_2yrs"] >= 1,
            tax_liens_flag = df["tax_liens"] >= 1,

            # shares
            s_actv_bc_tl = df["num_actv_bc_tl"] / (df["open_acc"] + 1e-6),
            s_bc_tl = df["num_bc_tl"] / (df["open_acc"] + 1e-6),
            s_il_tl = df["num_il_tl"] / (df["open_acc"] + 1e-6),
            s_rev_accts = df["num_rev_accts"] / (df["open_acc"] + 1e-6),

            # interactions
            int_rate_x_duration = df["int_rate"] * df["loan duration"], # higher rates are even riskier on 60 vs 36
            dti_x_util = df["dti"] * (df["revol_util"] / 100.0), # debt burden (DTI) is more problematic if utilization of their cards/lines is also high
            revol_bal_income_ratio = df["revol_bal"] / (df["annual_inc"] + 1e-6), # leverage: outstanding revolving balance / income
            fico_x_dti = df["fico_range_high"] * df["dti"], # same DTI can mean different risk depending on FICO score; "do they manage well or not?"
        )
    )

    return df_with_features

df_engineered = get_features(df_dropped)

In [ ]:
def categorical_encoding(df: pd.DataFrame) -> pd.DataFrame:
  """Encodings of categorical variables."""

  df_encoded = df.copy()

  # grade to numeric
  grade_map = {c: i+1 for i, c in enumerate(string.ascii_uppercase[:7])}
  df_encoded["grade_num"] = df_encoded["grade"].map(grade_map)

  # sub_grade to numeric
  sg = df_encoded["sub_grade"].astype(str).str.upper().str.strip()
  letter = sg.str[0]
  number = pd.to_numeric(sg.str[1:].str.extract(r"(\d+)", expand=False), errors="coerce")
  letter_map = {ch: i+1 for i, ch in enumerate("ABCDEFG")}
  base = letter.map(letter_map)
  sub_grade_num = (base - 1) * 5 + number
  df_encoded["sub_grade_num"] = sub_grade_num.astype("float32")

  # emp_length to numeric; map prob cleanest; maybe 10+ different?
  emp_length_map = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10
  }

  df_encoded["emp_length_num"] = df_encoded["emp_length"].map(emp_length_map).astype("float32")

  # one-hot
  onehot_cols = ["home_ownership", "purpose", "emp_title"]
  df_encoded = pd.get_dummies(df_encoded, columns=onehot_cols, prefix=onehot_cols, drop_first=True)

  # drop originals
  df_encoded = df_encoded.drop(columns=["grade", "sub_grade", "emp_length"])

  return df_encoded

df_encoded = categorical_encoding(df_engineered)

# Surrogate Model

In [ ]:
context = ["id", "Predictions", "target"]
y = ["Predicted probabilities"]
X = [col for col in df_encoded.columns if col not in (context + y)]